In [ ]:
import pandas as pd
import numpy as np
import pathlib
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

path_dict = {
    'train_data_dir_normal': r"C:\Users\beher\OneDrive\Desktop\seminar\X-AI-assisted-deep-learning-methods-to-detect-pneumonia-main\X-AI-assisted-deep-learning-methods-to-detect-pneumonia-main\chest_xray\train\NORMAL",
    'train_data_dir_pneumonia': r"C:\Users\beher\OneDrive\Desktop\seminar\X-AI-assisted-deep-learning-methods-to-detect-pneumonia-main\X-AI-assisted-deep-learning-methods-to-detect-pneumonia-main\chest_xray\train\PNEUMONIA",
    'test_data_dir_normal': r"C:\Users\beher\OneDrive\Desktop\seminar\X-AI-assisted-deep-learning-methods-to-detect-pneumonia-main\X-AI-assisted-deep-learning-methods-to-detect-pneumonia-main\chest_xray\test\NORMAL",
    'test_data_dir_pneumonia': r"C:\Users\beher\OneDrive\Desktop\seminar\X-AI-assisted-deep-learning-methods-to-detect-pneumonia-main\X-AI-assisted-deep-learning-methods-to-detect-pneumonia-main\chest_xray\test\PNEUMONIA",
    'data_val_dir_normal': r"C:\Users\beher\OneDrive\Desktop\seminar\X-AI-assisted-deep-learning-methods-to-detect-pneumonia-main\X-AI-assisted-deep-learning-methods-to-detect-pneumonia-main\chest_xray\val\NORMAL",
    'data_val_dir_pneumonia': r"C:\Users\beher\OneDrive\Desktop\seminar\X-AI-assisted-deep-learning-methods-to-detect-pneumonia-main\X-AI-assisted-deep-learning-methods-to-detect-pneumonia-main\chest_xray\val\PNEUMONIA"
}

for key, value in path_dict.items():
    path_dict[key] = pathlib.Path(value)

X = []
y = []

# Define the new image size (e.g., 128x128 pixels)
new_image_size = (128, 128)

for key, value in path_dict.items():
    images = list(path_dict[key].glob('*.jpeg'))
    for img in images:
        image = cv2.imread(str(img))
        resized_img = cv2.resize(image, new_image_size)  # Resize to the new size
        X.append(resized_img)
        if 'normal' in key:
            y.append(0)  # 0 - normal
        else:
            y.append(1)  # 1 - pneumonia

X = np.array(X)
y = np.array(y)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

X_train_scaled = X_train / 255
X_test_scaled = X_test / 255


y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


# Create a VGG16-based model
base_model = VGG16(include_top=False, weights='imagenet', input_shape=new_image_size + (3,))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

model = keras.Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import plotly.express as px

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Added EarlyStopping callback to stop training if validation loss doesn't improve for 10 epochs
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

model.fit(X_train_scaled, y_train, epochs=30, validation_data=(X_test_scaled, y_test), callbacks=[early_stopping])

model.evaluate(X_test_scaled, y_test)

# Evaluate the model on the test set
y_pred = model.predict(X_test_scaled)

# Convert predictions to binary values
y_pred_binary = np.argmax(y_pred, axis=1)
y_test_binary = np.argmax(y_test, axis=1)

# Calculate and print metrics
accuracy = accuracy_score(y_test_binary, y_pred_binary)
precision = precision_score(y_test_binary, y_pred_binary)
recall = recall_score(y_test_binary, y_pred_binary)
f1 = f1_score(y_test_binary, y_pred_binary)


print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")


# Plot ROC curve
fpr, tpr, _ = roc_curve(y_test_binary, y_pred[:, 1])
roc_auc = auc(fpr, tpr)


fig = px.area(
    x=fpr, y=tpr,
    title=f'ROC Curve (AUC={roc_auc:.4f})',
    labels=dict(x='False Positive Rate', y='True Positive Rate'),
    width=500, height=500
)

fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)

fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')
fig.show()

y_pred_scaled = np.argmax(y_pred, axis=1)
y_test_scaled = np.argmax(y_test, axis=1)

conf_matrix = confusion_matrix(y_test_scaled, y_pred_scaled)
sns.heatmap(conf_matrix, annot=True, fmt='d')


import lime
from lime import lime_image
from skimage.segmentation import mark_boundaries


def explain_image(image, index):
    explainer = lime_image.LimeImageExplainer()
    explanation = explainer.explain_instance(image.astype('double'), model.predict, top_labels=5, hide_color=0, num_samples=1000)
    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)
    plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))
    plt.title('Predicted - ' + str('Normal' if y_pred_scaled[index] == 0 else 'Pneumonia') +
              '\n Ground Truth - ' + str('Normal' if y_test_scaled[index] == 0 else 'Pneumonia') +
              ' \n Green Regions -> Supporting the predicted label \n Red Regions -> Against the predicted label')
    plt.show()

for index, img in enumerate(X_test_scaled[:10]):
    explain_image(img, index)





